In [1]:
import numpy as np
import pandas as pd
import pubchempy as pc
import scipy as sp

#rdkit imports
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import Descriptors
from rdkit.Chem import rdFMCS
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Avalon.pyAvalonTools import GetAvalonFP

## what do i want this model function to do?

I want it to...

1) check to see if the compound is a known enzymatic product
    * if so: return the enzyme/reaction/compound/starting compound
2) if not: pair the input or found compound with all the promiscuous products

3) calculate the distance between the query compound and each product of each promiscuous enzyme (NOT the average distance, just the distance)

4) return the n most similar reactions/substrates/whatever (number n is selected by user, default is hard-coded)

this code needs to do 2-4, 1 is external but needs to talk to this code

inputs: master_df, enzyme_col, pubchem_col, smiles_col, pubchem_cid
need to drop fingerprint and mol from columns if they are there
need to make sure this is the promsicuous master_df (positive only)
this is almost exactly the same inputs as pair_query_compound

____

preparing development dataframe

In [2]:
prom_pos = pd.read_csv('../../../big-datasets/positive-with-dist.csv')

In [4]:
prom_pos.drop(columns=['Fingerprint'], inplace=True)
prom_pos.head()

,enzyme,product,reacts,PubChemID,SMILES,n_C,n_H,n_O,n_N,n_P,n_S,n_X,n_DoU,MW,Dist
0,EC-1.14.14.77,2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE,1.0,5280483,CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(...,31,46,2,0,0,0,0,9,450.707,1.000000
1,EC-1.14.14.80,CPD-10515,1.0,25201835,CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O,18,35,4,0,0,0,0,1,315.474,0.714718
2,EC-1.14.14.80,PALMITATE,1.0,504166,CCCCCCCCCCCCCCCC(=O)[O-],16,31,2,0,0,0,0,1,255.422,0.714718
3,EC-1.14.14.80,OLEATE-CPD,1.0,5460221,CCCCCCCCC=CCCCCCCCC(=O)[O-],18,33,2,0,0,0,0,2,281.460,0.714718
4,EC-1.14.14.80,STEARIC_ACID,1.0,3033836,CCCCCCCCCCCCCCCCCC(=O)[O-],18,35,2,0,0,0,0,1,283.476,0.714718


In [5]:
def fingerprint_products(input_df): #fingerprints all products in a given df
    '''DocString'''
    
    mol_list = []
    fp_list = []
    
    for index, row in input_df.iterrows():
        mol_list.append(Chem.rdmolfiles.MolFromSmiles(row['SMILES'])) #get mols from SMILES and add mols to list
        fp_list.append(FingerprintMols.FingerprintMol(Chem.rdmolfiles.MolFromSmiles(row['SMILES']))) #get fingerprints from mols and and fingerprints to list
        
    input_df.insert(6, column='Mol', value=mol_list)
    input_df.insert(7, column='Fingerprint', value= fp_list)
            
    return input_df

In [6]:
prom_pos = fingerprint_products(prom_pos)

______

In [18]:
def cid_to_smiles(cid):

    try:
        compound = pc.get_compounds(cid)[0]
        smiles = compound.canonical_smiles
    except BaseException:
        pass
    
    return smiles, cid


def dist_for_expansion(prom_df, pubchem_cid, num_similar):
    
    #####this probably isn't necessary because in the final version, this will all be run in one sitting and not saved and loaded etc
    #if 'Fingerprint' and 'Mol' in prom_df.columns:
     #   prom_df = prom_df.drop(columns=['Fingerprint','Mol'])
    #else:
     #   continue
    
    bad_cols = ['ExpanDist']
    
    for col in bad_cols:
        if col in prom_df.columns:
            prom_df.drop(columns=col, inplace=True)
        else:
            continue
    
        
    smiles, _ = cid_to_smiles(pubchem_cid)
    
    if len(smiles) == 0:
        raise 'query compound SMILES string could not be retrieved'
    else:
        pass
    
    mol = Chem.rdmolfiles.MolFromSmiles(smiles)
    fingerprint = FingerprintMols.FingerprintMol(mol)
    
    
    prom_df['ExpanDist'] = ''
    
    for index, row in prom_df.iterrows():
        comp = DataStructs.FingerprintSimilarity(row['Fingerprint'], fingerprint, metric=DataStructs.TanimotoSimilarity)
        prom_df['ExpanDist'].loc[index] = comp

    prom_df.sort_values(by='ExpanDist', ascending=False, inplace=True)
    
    if num_similar == 'None':
        n = 20
    else:
        n = num_similar
    
    selected = prom_df.iloc[:n,:].copy()
    
    return selected

In [19]:
small = prom_pos.iloc[:20,:].copy()

In [20]:
dist_for_expansion(small, 243, 5)

C:\Users\ellie\Miniconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,enzyme,product,reacts,PubChemID,SMILES,n_C,Mol,Fingerprint,n_H,n_O,n_N,n_P,n_S,n_X,n_DoU,MW,Dist,ExpanDist
7,EC-1.8.1.20,HYPOTAURINE,1.0,25244088,C(CS(=O)[O-])[NH3+],2,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, ...",7,2,1,0,1,0,0,109.150,1.000000,0.403509
3,EC-1.14.14.80,OLEATE-CPD,1.0,5460221,CCCCCCCCC=CCCCCCCCC(=O)[O-],18,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",33,2,0,0,0,0,2,281.460,0.714718,0.375
1,EC-1.14.14.80,CPD-10515,1.0,25201835,CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O,18,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",35,4,0,0,0,0,1,315.474,0.714718,0.357143
0,EC-1.14.14.77,2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE,1.0,5280483,CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(...,31,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...",46,2,0,0,0,0,9,450.707,1.000000,0.350806
10,EC-1.8.3.2,CPD-17049,1.0,90658023,C1=CC=C(C=C1)CC2(C(=O)NC(C(=O)N2)(CO)S)S,12,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, ...",14,3,2,0,2,0,7,298.389,1.000000,0.346457


In [21]:
dist_for_expansion(small, 243, 'None')

C:\Users\ellie\Miniconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,enzyme,product,reacts,PubChemID,SMILES,n_C,Mol,Fingerprint,n_H,n_O,n_N,n_P,n_S,n_X,n_DoU,MW,Dist,ExpanDist
7,EC-1.8.1.20,HYPOTAURINE,1.0,25244088,C(CS(=O)[O-])[NH3+],2,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, ...",7,2,1,0,1,0,0,109.150,1.000000,0.403509
3,EC-1.14.14.80,OLEATE-CPD,1.0,5460221,CCCCCCCCC=CCCCCCCCC(=O)[O-],18,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",33,2,0,0,0,0,2,281.460,0.714718,0.375
1,EC-1.14.14.80,CPD-10515,1.0,25201835,CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O,18,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",35,4,0,0,0,0,1,315.474,0.714718,0.357143
0,EC-1.14.14.77,2-METHYL-3-PHYTYL-14-NAPHTHOQUINONE,1.0,5280483,CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(...,31,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...",46,2,0,0,0,0,9,450.707,1.000000,0.350806
10,EC-1.8.3.2,CPD-17049,1.0,90658023,C1=CC=C(C=C1)CC2(C(=O)NC(C(=O)N2)(CO)S)S,12,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, ...",14,3,2,0,2,0,7,298.389,1.000000,0.346457
17,EC-1.14.14.101,S-N-METHYLCOCLAURINE,1.0,6971065,C[NH+]1CCC2=CC(=C(C=C2C1CC3=CC=C(C=C3)O)O)OC,18,<rdkit.Chem.rdchem.Mol object at 0x0000028FC53...,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...",22,3,1,0,0,0,8,300.378,1.000000,0.344
6,EC-2.4.1.241,CPD-14553,1.0,9549262,C1=CN(C(=O)NC1=O)C2C(C(C(O2)COP(=O)([O-])OP(=O...,15,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, ...",22,17,2,2,0,0,7,564.286,1.000000,0.34375
8,EC-4.3.2.10,P-RIBOSYL-4-SUCCCARB-AMINOIMIDAZOLE,1.0,45266647,C1=NC(=C(N1C2C(C(C(O2)COP(=O)([O-])[O-])O)O)N)...,13,<rdkit.Chem.rdchem.Mol object at 0x0000028FC52...,"[1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, ...",15,12,4,1,0,0,9,450.253,1.000000,0.34375
14,EC-2.7.7.81,CTP,1.0,7058166,C1=CN(C(=O)N=C1N)C2C(C(C(O2)COP(=O)([O-])OP(=O...,9,<rdkit.Chem.rdchem.Mol object at 0x0000028FC53...,"[1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, ...",12,14,3,3,0,0,7,479.124,0.290483,0.34375
13,EC-2.1.1.237,CPD-13823,1.0,56927772,CCC1C(C=CC=CC(=O)C(CC(C(C(C=CC(=O)O1)C)OC2C(C(...,35,<rdkit.Chem.rdchem.Mol object at 0x0000028FC53...,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, ...",58,11,1,0,0,0,7,668.845,1.000000,0.341176
